In [ ]:
import csv
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
import lda
import numpy as np
from pylab import *


corpus = []
jieba.set_dictionary('dict.txt.big')

with open('Output.csv', newline='',encoding='UTF-8') as csvfile:
    
    rows = csv.reader(csvfile)
    
    for row in rows:
        reHTTPS=re.compile(r'https://[a-zA-Z0-9.?/&=:]*',re.S)
        noHTTPS=reHTTPS.sub("",row[2])
        
        reHTTP=re.compile(r'http://[a-zA-Z0-9.?/&=:]*',re.S)
        noHTTP=reHTTP.sub("",noHTTPS)
        
        fn = noHTTP.replace(r"\u3000", "").replace(r"\n","").replace("#","")

        corpus.append(" ".join(jieba.cut(fn.split(',')[0], cut_all=False)))
        
#print(corpus)

In [ ]:
def get_lda_input(chapters):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    return X.toarray(), vectorizer

def lda_train(weight, vectorizer):
    model = lda.LDA(n_topics=5, n_iter=500, random_state=1)
    model.fit(weight)

    doc_num = len(weight)
    topic_word = model.topic_word_
    vocab = vectorizer.get_feature_names()
    titles = ["第{}章".format(i) for i in range(1, doc_num + 1)]

    n_top_words = 10000
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words + 1):-1]
        #print('Topic {}: {}'.format(i, ' '.join(topic_words)))
        with open('Topic{}.txt'.format(i),'a',encoding='UTF-8') as Topics:
            Topics.write(' '.join(topic_words))

In [ ]:
weight, vectorizer = get_lda_input(corpus)
lda_train(weight, vectorizer)